# Carga Incremental con Auto Loader en Databricks

## ¿Qué es Auto Loader?

Auto Loader es una herramienta de Databricks que permite cargar datos de manera automática e incremental desde almacenamiento de objetos en la nube (como S3, Azure Blob Storage o Google Cloud Storage) hacia tablas Delta Lake.

### ¿Por qué usar Auto Loader?

Auto Loader tiene sentido usarlo en combinación con herramientas de extracción y carga de datos como **Fivetran** o **dltHub**. Estas herramientas se encargan de:

1. **Extraer datos** desde sistemas origen (bases de datos, APIs, etc.)
2. **Depositar archivos** en un directorio de almacenamiento de objetos
3. **Auto Loader** se encarga de detectar automáticamente estos archivos nuevos y cargarlos incrementalmente a Databricks

En nuestro caso específico, estamos usando un **script automatizado sencillo** que extrae datos diariamente desde una base de datos PostgreSQL. El script implementa una **carga incremental por cursor** usando la columna `purchase_date` como cursor para detectar cambios:

```sql
-- Extraer datos de ayer
extract_date = (datetime.now(timezone.utc).date() - timedelta(days=1)).isoformat()
where_clause = f"WHERE purchase_date::date = DATE '{extract_date}'"

SELECT
    account_feature_id,
    account_id,
    feature_id,
    purchase_date,
    amount_paid
FROM public.account_premium_features
{where_clause}
```

El **cursor** en este caso es la columna `purchase_date`: cada día el script extrae solo los registros del día anterior, asegurando que no se reprocesen datos históricos y que la carga sea incremental.

Este patrón crea un **pipeline EL completo**:
- **Extract**: Script automatizado extrae datos diarios
- **Load**: Archivos CSV se depositan en el directorio de landing. Auto Loader detecta y carga incrementalmente a Delta Lake

### Características principales de Auto Loader:

- **Detección automática de archivos nuevos**: Auto Loader monitorea continuamente el directorio de origen y detecta automáticamente cuando llegan archivos nuevos
- **Procesamiento incremental**: Solo procesa los archivos que no han sido procesados anteriormente, evitando reprocesar datos ya cargados
- **Manejo de esquemas**: Puede inferir el esquema automáticamente o usar esquemas evolucionados
- **Checkpointing**: Mantiene el estado del procesamiento para asegurar que no se pierdan datos en caso de fallos
- **Soporte para múltiples formatos**: CSV, JSON, Parquet, Avro, entre otros

### ¿Cómo detecta Auto Loader los archivos nuevos?

Auto Loader soporta dos modos principales para detectar archivos nuevos en el directorio de origen:

#### 1. **Directory Listing Mode** (Modo de listado de directorios)
- **Cómo funciona**: Auto Loader lista periódicamente el directorio de entrada para identificar archivos nuevos
- **Ventajas**: Fácil de configurar, no requiere permisos adicionales más allá del acceso a los datos
- **Limitaciones**: Puede ser menos eficiente en directorios muy grandes o con muchos archivos nuevos
- **Optimización**: En Databricks Runtime 9.1+, Auto Loader detecta automáticamente si los archivos llegan con orden léxico y reduce las llamadas a la API

#### 2. **File Notification Mode** (Modo de notificación de archivos - recomendado)
- **Cómo funciona**: Utiliza servicios de notificación y cola de la infraestructura cloud para suscribirse a eventos de archivos
- **Ventajas**: Más performante y escalable que el modo directory listing
- **Configuración**: Auto Loader puede configurar automáticamente el servicio de notificaciones si habilitas eventos de archivos en la ubicación externa
- **Recomendación**: Databricks recomienda este modo para la mayoría de las cargas de trabajo

**Nota importante**: Puedes cambiar entre modos de detección durante reinicios del stream manteniendo las garantías de procesamiento exactly-once.

### ¿Cómo funciona?

Auto Loader utiliza un patrón de **Structured Streaming** para procesar los archivos. Structured Streaming es el motor de procesamiento de streams de Apache Spark que permite tratar flujos de datos como si fueran tablas SQL.

En Structured Streaming, definimos una consulta que se ejecuta continuamente sobre datos que llegan en tiempo real. En el caso de Auto Loader, esta consulta:
1. Lee archivos desde el directorio de origen
2. Los procesa según el formato especificado
3. Escribe los resultados a una tabla Delta
4. Mantiene un checkpoint para recordar qué archivos ya fueron procesados

### Carga incremental basada en detección de cambios

En esta notebook implementamos una **carga incremental** que combina dos estrategias complementarias:

1. **Detección de archivos nuevos**: Auto Loader detecta automáticamente archivos nuevos en el directorio de origen
2. **Checkpointing de procesamiento**: Los checkpoints mantienen el estado de qué archivos ya fueron procesados

Esta carga se ejecuta en modo `availableNow=True`, procesando todos los datos disponibles en el momento y luego deteniéndose (no es un stream continuo).

Referencias:
- [Documentación oficial de Auto Loader](https://docs.databricks.com/aws/en/ingestion/cloud-object-storage/auto-loader/)
- [Guía de Structured Streaming](https://spark.apache.org/docs/latest/streaming/getting-started.html#programming-model)

## Configuración de la carga incremental

En esta sección configuramos los parámetros necesarios para la carga incremental. Utilizamos **widgets** de Databricks para hacer la notebook parametrizable y reutilizable.

### Parámetros configurables:

- **Alumno**: Identificador del estudiante (se utiliza para crear namespaces únicos)
- **Catálogo**: Catálogo de Databricks donde se creará la tabla destino
- **Volumen landing**: Directorio de origen donde llegan los archivos CSV nuevos
- **Volumen bronce**: Directorio de destino en la capa bronze del data lake
- **Tabla**: Nombre de la tabla que se creará en el catálogo

Los widgets permiten ejecutar la misma notebook con diferentes configuraciones sin modificar el código.

In [0]:
# Configuración de parámetros mediante widgets
# Los widgets permiten parametrizar la notebook para diferentes entornos y usuarios

# Creamos los widgets para configurar la carga
dbutils.widgets.text("alumno", "bruno", "Nombre del alumno")
dbutils.widgets.text("catalogo", "bronce_dev", "Catálogo de destino")
dbutils.widgets.text("volumen_landing", "/Volumes/landing_dev/postgres_datavision/landing-volume-postgres-datavision/", "Directorio de origen (landing)")
dbutils.widgets.text("volumen_bronce", "", "Directorio de destino (bronce)")
dbutils.widgets.text("tabla", "account_premium_features", "Nombre de la tabla a crear")

# Leemos los valores de los widgets
ALUMNO = dbutils.widgets.get("alumno")
CATALOGO = dbutils.widgets.get("catalogo")
VOLUMEN_LANDING = dbutils.widgets.get("volumen_landing")

# Si no se especifica volumen_bronce, lo construimos automáticamente
VOLUMEN_BRONCE_INPUT = dbutils.widgets.get("volumen_bronce")
if VOLUMEN_BRONCE_INPUT.strip() == "":
    VOLUMEN_BRONCE = f"/Volumes/{CATALOGO}/datavision/bronce-volume-datavision"
else:
    VOLUMEN_BRONCE = VOLUMEN_BRONCE_INPUT

TABLA = dbutils.widgets.get("tabla")

# Mostramos la configuración final
print(f"Configuración de carga:")
print(f"- Alumno: {ALUMNO}")
print(f"- Catálogo: {CATALOGO}")
print(f"- Volumen landing: {VOLUMEN_LANDING}")
print(f"- Volumen bronce: {VOLUMEN_BRONCE}")
print(f"- Tabla: {TABLA}")

## Ejecución de la carga incremental

En esta sección implementamos la **carga incremental** utilizando Auto Loader y Structured Streaming.

### ¿Qué hace este código?

1. **Checkpointing**: Creamos una ruta para guardar el estado del procesamiento. Los checkpoints son archivos que recuerdan qué archivos ya fueron procesados, permitiendo reanudar la carga desde donde se quedó.

2. **Lectura con Auto Loader**: Utilizamos `spark.readStream.format("cloudFiles")` para configurar Auto Loader:
   - `cloudFiles.format`: Especifica que los archivos son CSV
   - `cloudFiles.schemaLocation`: Define dónde guardar el esquema inferido automáticamente

3. **Escritura estructurada**: Configuramos la escritura del stream:
   - `checkpointLocation`: Mantiene el estado del procesamiento
   - `trigger(availableNow=True)`: Procesa todos los datos disponibles y se detiene (no continuo)
   - `toTable()`: Escribe directamente a una tabla Delta Lake

### Ventajas de este enfoque:

- **Incremental**: Solo procesa archivos nuevos
- **Fault-tolerant**: Los checkpoints permiten recuperarse de fallos
- **Escalable**: Puede manejar grandes volúmenes de datos
- **Automático**: Detecta nuevos archivos sin intervención manual

## Structured Streaming y procesamiento incremental

### ¿Qué es Structured Streaming?

Structured Streaming es el framework de Apache Spark para procesar datos en tiempo real de manera estructurada. A diferencia de otros sistemas de streaming que procesan eventos uno por uno, Structured Streaming trata los flujos de datos como si fueran **tablas SQL infinitas** que se actualizan continuamente.

**Modelo de programación de Structured Streaming:**
- Definimos una consulta SQL-like sobre un flujo de datos
- Spark se encarga de ejecutar la consulta de manera incremental y eficiente
- Los resultados se actualizan automáticamente cuando llegan nuevos datos

### Checkpointing en Structured Streaming

En el contexto de Structured Streaming, el **checkpointing** es fundamental para la carga incremental:

1. **Checkpoint**: Un marcador persistente que indica qué archivos ya fueron procesados
2. **Recuperación de fallos**: Permite reanudar el procesamiento desde el último punto exitoso
3. **Consistencia**: Garantiza que cada archivo se procesa exactamente una vez

**Ventajas del checkpointing:**
- **Eficiencia**: No reprocesamos datos ya cargados
- **Confiabilidad**: Los checkpoints permiten recuperarse de fallos
- **Escalabilidad**: Puede manejar grandes volúmenes sin degradación del rendimiento
- **Simplicidad**: El framework maneja automáticamente la complejidad del estado

### Cómo funciona en Auto Loader:

1. **Detección**: Auto Loader monitorea el directorio de origen
2. **Procesamiento**: Solo procesa archivos nuevos desde el último checkpoint
3. **Estado**: Actualiza el checkpoint después de procesar exitosamente
4. **Consistencia**: Garantiza que cada archivo se procesa exactamente una vez

Esta combinación de Auto Loader + Structured Streaming nos da una solución robusta y eficiente para cargas incrementales.

In [0]:
from pyspark.sql.functions import current_date

# Configuración del checkpoint para mantener el estado del procesamiento
# El checkpoint es crucial para la carga incremental - guarda qué archivos ya fueron procesados
checkpoint_path = f"{VOLUMEN_BRONCE}/_checkpoint_{ALUMNO}/{TABLA}"

# Iniciamos la consulta de Structured Streaming con Auto Loader
# Esta es la configuración principal de la carga incremental
query = (spark.readStream
  # Especificamos que usaremos el formato cloudFiles (Auto Loader)
  .format("cloudFiles")

  # Configuramos el formato de los archivos de origen (CSV en este caso)
  .option("cloudFiles.format", "csv")

  # Especificamos dónde guardar la información del esquema inferido
  # Esto permite manejar cambios en el esquema de manera automática
  .option("cloudFiles.schemaLocation", checkpoint_path)

  # Definimos el directorio de origen donde llegan los archivos nuevos
  .load(f"{VOLUMEN_LANDING}/{TABLA}/")
  .withColumn("fecha_carga", current_date())

  # Configuramos la escritura de la stream
  .writeStream

  # Especificamos dónde guardar los checkpoints del procesamiento
  # Los checkpoints contienen el estado de qué archivos ya fueron procesados
  .option("checkpointLocation", checkpoint_path)

  # Usamos trigger availableNow para procesar todos los datos disponibles
  # y luego detener la consulta (no es un stream continuo)
  .trigger(availableNow=True)

  # Especificamos la tabla destino donde se escribirán los datos
  .toTable(f"{CATALOGO}.datavision_{ALUMNO}.{TABLA}"))

# La consulta se ejecuta automáticamente cuando se define con toTable()
# Esto inicia el procesamiento incremental de los archivos

## Verificación de la carga

Después de ejecutar la carga incremental, es importante verificar que los datos se cargaron correctamente. La consulta SQL a continuación cuenta los registros en la tabla destino.

In [0]:
%sql
SELECT count(*) FROM bronce_dev.datavision_nicolas_herrera.account_premium_features

# Ejercicio Práctico: Experimentando con Checkpointing en Auto Loader

## Objetivo

Ahora que entendiste cómo funciona Auto Loader, el checkpointing y la carga incremental, vamos a realizar **dos experimentos controlados** para analizar el comportamiento del sistema y extraer conclusiones importantes sobre cómo maneja el estado y la detección de cambios.

## Experimento 1: Recrear la tabla y ejecutar nuevamente

### Consigna:
1. **Ejecuta primero la notebook completa** con la configuración por defecto para cargar la tabla `account_premium_features`
2. **Verifica la cantidad de registros** cargados inicialmente
3. **Borra la tabla** con este comando SQL:
   ```sql
   DROP TABLE IF EXISTS {CATALOGO}.datavision_{ALUMNO}.account_premium_features;
   ```
4. **Ejecuta la notebook nuevamente** (sin modificar ningún parámetro). La tabla se debe recrear automaticamente.
5. **Compara los resultados**:
   - ¿Se cargaron los mismos registros?
   - ¿Cuántos registros hay ahora en la tabla?
   - ¿Qué conclusiones puedes extraer sobre el comportamiento del checkpoint?

## Experimento 2: Borrar el checkpoint y ejecutar nuevamente

### Consigna:
1. **Ejecuta primero la notebook completa** nuevamente
2. **Verifica la cantidad de registros** actuales
3. **Borra el directorio de checkpoint** ejecutando este comando:
   ```python
   checkpoint_path = f"{VOLUMEN_BRONCE}/_checkpoint_{ALUMNO}/account_premium_features"
   dbutils.fs.rm(checkpoint_path, recurse=True)
   print(f"Checkpoint borrado: {checkpoint_path}")
   ```
4. **Ejecuta la notebook nuevamente** (sin modificar ningún parámetro)
5. **Compara los resultados**:
   - ¿Se duplicaron los datos?
   - ¿Cuántos registros hay ahora en la tabla?
   - ¿Qué sucedió con el procesamiento?

## Análisis y conclusiones

Después de completar ambos experimentos, responde estas preguntas:

### Experimento 1 (Recrear tabla):
- ¿Por qué al recrear la tabla se volvieron a cargar todos los datos?
- ¿Qué rol juega la existencia de la tabla en el proceso de carga?
- ¿Cómo se comporta Auto Loader cuando la tabla destino ya existe vs cuando no existe?

### Experimento 2 (Borrar checkpoint):
- ¿Qué sucedió cuando borramos el checkpoint?
- ¿Se duplicaron los datos o se reprocesaron los archivos?
- ¿Cuál es la diferencia entre "checkpoint" y "estado de la tabla"?
- ¿Por qué es importante el checkpoint para la carga incremental?

### Conclusiones generales:
- ¿Cuál es la relación entre el checkpoint y la detección de archivos ya procesados?
- ¿En qué escenarios sería peligroso borrar un checkpoint?
- ¿Cómo garantiza exactamente-once processing el sistema de checkpointing?
- ¿Qué ventajas tiene este mecanismo comparado con un sistema sin checkpoints?